In [12]:
from sklearn.decomposition import PCA
import pandas as pd
import os
from spotipy.oauth2 import SpotifyClientCredentials
from sqlalchemy import create_engine
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
import spotipy
import getpass
import pickle

In [ ]:
c_id = getpass.getpass()
c_se = getpass.getpass()
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se, requests_timeout=1000))

In [3]:
pw = 'password1999'
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string)

In [23]:
df_feat = pd.read_sql("SELECT * FROM spotify.df_complet", engine)
X = df_feat.iloc[:,12:].iloc[:,0:11]
X = X.drop_duplicates(keep='first')

In [24]:
scaler = StandardScaler()
scaler.fit(X)

StandardScaler()

In [25]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)

In [26]:
with open("scaler_3000_178k.pickle", "wb") as f:
    pickle.dump(scaler,f)

In [27]:
kmeans = KMeans(n_clusters=3000, init="k-means++",n_init=5,random_state=128)
kmeans.fit(X_scaled_df)

KMeans(n_clusters=3000, n_init=5, random_state=128)

In [28]:
with open("kmeans_3000_178k.pickle", "wb") as f:
    pickle.dump(kmeans,f)

### Database scaling

In [29]:
def scale_database():
    df_feat = pd.read_sql("SELECT * FROM spotify.df_complet", engine)
    X = df_feat.iloc[:,12:].iloc[:,0:11]
    X = X.drop_duplicates(keep='first')
    scaler = StandardScaler()
    scaler.fit(X)
    scaler.transform(X)
    X_scaled = scaler.transform(X)
    X_scaled_df_proto = pd.DataFrame(X_scaled, columns = X.columns)
    return X_scaled_df_proto

In [36]:
def model_database(X_scaled_df_proto):
    model = load("kmeans_3000_178k.pickle")
    clusters = model.predict(X_scaled_df_proto)
    return clusters

In [31]:
def add_clusters_merge (clusters):
    X_scaled_df_proto['cluster'] = clusters
    df_feat_unique = df_feat.drop_duplicates(subset=['danceability','energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],keep='first')
    df_feat_final = df_feat_unique.reset_index().merge(X_scaled_df_proto.reset_index(), left_index=True, right_index=True)
    return df_feat_final

In [34]:
def load(file_name):
    try:
        with open(file_name, "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        print("File not found!")

In [ ]:
df_feat = pd.read_sql("SELECT * FROM spotify.df_complet", engine)

In [37]:
X_scaled_df_proto = scale_database()
    
clusters = model_database(X_scaled_df_proto)
    
df_feat_final = add_clusters_merge (clusters)

df_feat_final.to_sql("df_feat_final", connection_string, schema='spotify', if_exists='replace', index = False)

144853